In [4]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [77]:
rev1=pd.read_csv("/media/jyotiraditya/Ultra Touch/repos/SiteReports/data/common_data/incompelete_brands_reviews_data_set_1.csv")
rev2=pd.read_csv("/media/jyotiraditya/Ultra Touch/repos/SiteReports/notebooks/gmaps_reviews_per_day_remaining.csv")
brands=pd.concat([rev1,rev2]).reset_index()
brands=brands.replace({np.Infinity:None})

In [78]:
def avg(arr):
    return sum(arr)/len(arr)

In [79]:
brands[brands['num_reviews_per_day']==np.Infinity].iloc[0]['data']

IndexError: single positional indexer is out-of-bounds

In [80]:
a=(brands.fillna({"num_reviews_per_day":0}).astype({"num_reviews_per_day":np.float128}).query("brand_id=='lbf'")['num_reviews_per_day'].values.tolist())
a

/tmp/ipykernel_359946/992413877.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  a=(brands.fillna({"num_reviews_per_day":0}).astype({"num_reviews_per_day":np.float128}).query("brand_id=='lbf'")['num_reviews_per_day'].values.tolist())


[3.429861174013423497,
 0.011284797142187800653,
 2.2717654432566809497,
 0.71947535790485273033,
 4.216533485749458521,
 0.25092127202646979933,
 0.84018178684494593256,
 1.7463542452655793635,
 0.003135274351202299841,
 2.7460243688580754373,
 0.13617687695493199684,
 2.9700176338625929873,
 0.13229000231362450513,
 0.028345909817004598386,
 0.74059052589674378986,
 1.1655406311923051099,
 0.0,
 2.095155629477234882,
 0.12884662024860418783,
 0.22974371973303880923,
 1.7295569119428528815,
 0.60332024122675964595,
 1.6673819081661216401,
 0.112946669716647199455,
 0.027923265315620098698,
 0.035492387686452096796,
 0.19584494974342880891,
 0.75808043683713122185,
 0.01353860830314690053,
 0.025024982695108600322,
 2.0856683444548864692,
 0.074312411707698103336,
 0.0010101189720229000572,
 0.037973971567474500144,
 0.5147982639447034048,
 0.124476267690284095324,
 1.3440682913733164039,
 0.08970800802634880389,
 0.2669775398420906054,
 0.051821961611001297887,
 0.01494545726426189935

In [81]:
brands.fillna({"num_reviews_per_day":0}).groupby("brand_id").agg(avg_reviews_per_day_city=("num_reviews_per_day",np.average)).query("brand_id=='lbf'")

/tmp/ipykernel_359946/3201538897.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  brands.fillna({"num_reviews_per_day":0}).groupby("brand_id").agg(avg_reviews_per_day_city=("num_reviews_per_day",np.average)).query("brand_id=='lbf'")


,avg_reviews_per_day_city
brand_id,
lbf,0.695162


In [8]:
brands = pd.read_csv(
        "/media/jyotiraditya/Ultra Touch/repos/SiteReports/data/common_data/bangalore/raw/location_poi_data.csv")
brands = brands.query("brand_id!='N_A'")
brands.shape

(27648, 15)

In [12]:
rev.columns

Index(['index', 'id', 'name', 'lat', 'lng', 'top_category', 'sub_categories',
       'type', 'brand_name', 'brand_id', 'address', 'pincode', 'town_name',
       'source', 'number_of_votes', 'rating', 'created_at', 'verified',
       'last_verification_date', 'active', 'additional_data', 'geo_filter_1',
       'geo_filter_2', 'service_options', 'oldest_review_timestamp',
       'top_reviews', 'top_review_keywords', 'price_level', 'category',
       'city_id', 'hex_id', 'lambda_url', 'data', 'updated_num_reviews',
       'current_time', 'num_days', 'num_reviews_per_day', 'statusCode',
       'request_params', 'error'],
      dtype='object')

In [108]:
def get_top_brands():
    # brands = pd.read_csv(
    #     "/media/jyotiraditya/Ultra Touch/repos/SiteReports/data/common_data/bangalore/raw/location_poi_data.csv")
    # brands = brands.query("brand_id!='N_A'")
    rev1=pd.read_csv("/media/jyotiraditya/Ultra Touch/repos/SiteReports/data/common_data/incompelete_brands_reviews_data_set_1.csv")
    rev2=pd.read_csv("/media/jyotiraditya/Ultra Touch/repos/SiteReports/notebooks/gmaps_reviews_per_day_remaining.csv")
    brands=pd.concat([rev1,rev2]).query("brand_id!='N_A'").reset_index().fillna({"num_reviews_per_day":0})
    brands=brands.replace({np.Infinity:None})
    brands_rev=brands.fillna({"num_reviews_per_day":0}).groupby("brand_id").agg(avg_reviews_per_day_city=("num_reviews_per_day",np.mean))
    brands = gpd.GeoDataFrame(brands, geometry=gpd.points_from_xy(brands['lng'], brands['lat']))
    malls = pd.read_csv("/media/jyotiraditya/Ultra Touch/repos/SiteReports/data/common_data/bng_malls_ranked.csv")
    malls = gpd.GeoDataFrame(malls, geometry=gpd.GeoSeries.from_wkt(malls['wkt']))[['id','name', 'geometry']]
    malls['location_type']='mall'
    clusters=pd.read_csv("/media/jyotiraditya/Ultra Touch/repos/SiteReports/data/common_data/blr_cluster_geom.csv")
    clusters = gpd.GeoDataFrame(clusters, geometry=gpd.GeoSeries.from_wkt(clusters['polygon_updated']))[['micro_market_clusters_id','cluster_name', 'geometry']]
    clusters = clusters.rename(columns={"micro_market_clusters_id":"id","cluster_name":"name"})
    clusters['location_type']='market_cluster'
    other=pd.read_csv("/media/jyotiraditya/Ultra Touch/repos/SiteReports/data/common_data/bangalore/raw/location_pincode_locality_data.csv")
    other = gpd.GeoDataFrame(other, geometry=gpd.GeoSeries.from_wkt(other['geometry']))
    stores=pd.concat([malls,clusters,other]).reset_index(drop=True)
    joined = gpd.sjoin(stores, brands, how="left", predicate='contains')
    joined=joined.merge(brands_rev,how='left',on='brand_id')
    joined1 = joined.groupby(["id_left",'name_left', "category","location_type"]).agg(stores_count=('brand_id', lambda x: np.count_nonzero(x)),
                                                         unique_brands_count=('brand_id', lambda x: len(set(list(x)))),
                                                         brand_names=('brand_name', lambda x:list(set(x))),
                                                         avg_num_reviews_per_day=("num_reviews_per_day",np.mean),
                                                         )
    joined2 = joined.groupby(["id_left",'name_left', 'category',"brand_name","location_type",'avg_reviews_per_day_city']).agg(avg_num_reviews_per_day=("num_reviews_per_day",np.mean)).reset_index()
    
    joined2['avg_rev_per_day_difference']=joined2['avg_reviews_per_day_city']-joined2['avg_num_reviews_per_day']
    joined1=joined1.reset_index().rename(columns={"id_left":"id","name_left":"name"})
    joined2=joined2.reset_index().rename(columns={"id_left":"id","name_left":"name"})
    joined2=joined2.sort_values(by='avg_rev_per_day_difference',ascending=False)
    joined2['brand_reviews_per_day']=joined2[['brand_name','avg_rev_per_day_difference']].apply(lambda x:{x[0]:x[1]},axis=1)
    joined2=joined2.groupby(['id',"category",'location_type']).agg(top_brands=('brand_reviews_per_day',lambda x:list((x)))).reset_index()
    joined1=joined1.merge(joined2,how='left',on=['id','location_type','category'])
    
    return joined1,joined2

In [107]:
get_top_brands()[1]

/tmp/ipykernel_359946/3002362798.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  brands_rev=brands.fillna({"num_reviews_per_day":0}).groupby("brand_id").agg(avg_reviews_per_day_city=("num_reviews_per_day",np.mean))
/tmp/ipykernel_359946/3002362798.py:9: FutureWarning: The provided callable <function mean at 0x7fbcb2baf250> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  brands_rev=brands.fillna({"num_reviews_per_day":0}).groupby("brand_id").agg(avg_reviews_per_day_city=("num_reviews_per_day",np.mean))
/tmp/ipykernel_359946/3002362798.py:23: FutureWarning: The provided callable <function mean at 0x7fbcb2baf250> is currently using SeriesGroupBy

top_brands
id                   category         location_type                                                    
0                    animal           market_cluster  [{'snouters': 0.015451413046746076}, {'Sanchu ...
                     bar_pub          market_cluster               [{'thepub1522': 0.9223239079616672}]
                     clinic           market_cluster  [{'Lenskart': 0.7615487573784313}, {'Clove Den...
                     clothing_store   market_cluster  [{'Levi': 0.008359037491394057}, {'Just Casual...
                     coffee_shop      market_cluster  [{'Lassi Shop': 0.11430268807797984}, {'Starbu...
...                                                                                                 ...
ffb2cc08b03274e077cc food_other       neighbourhood                  [{'Winni': -0.041875894951886955}]
ffbf68d1f45c975a4f3a food_other       neighbourhood             [{'Juice Junction': 0.103963340149974}]
ffd5be412931cbf39028 clinic           neighbourhood                    [{'ESIC': 0.025170441957978094}]
                     clothing_store   neighbourhood          [{'Reliance Trends': 0.20477224635008134}]
                     electronic_store neighbourhood   [{'Girias': 0.061927720229659686}, {'sbestores...

[17883 rows x 1 columns]

In [110]:
get_top_brands()[0].to_csv("brands category mix.csv",index=False)

/tmp/ipykernel_359946/3527490874.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  brands_rev=brands.fillna({"num_reviews_per_day":0}).groupby("brand_id").agg(avg_reviews_per_day_city=("num_reviews_per_day",np.mean))
/tmp/ipykernel_359946/3527490874.py:9: FutureWarning: The provided callable <function mean at 0x7fbcb2baf250> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  brands_rev=brands.fillna({"num_reviews_per_day":0}).groupby("brand_id").agg(avg_reviews_per_day_city=("num_reviews_per_day",np.mean))
/tmp/ipykernel_359946/3527490874.py:23: FutureWarning: The provided callable <function mean at 0x7fbcb2baf250> is currently using SeriesGroupBy